# Using EloModel Class

In [1]:
import pandas as pd
import numpy as np

from elo_model import EloModel
from elo_model import evaluate_EloModel
from data_forms import TrainingData
from heuristics import probabilistic_elo_init as init_heur1

DATA_PATH = "data/atp_matches_"
atp_2022_df = pd.read_csv(DATA_PATH + '2022.csv')
atp_2023_df = pd.read_csv(DATA_PATH + '2023.csv')

matches_22to23 = TrainingData([atp_2022_df, atp_2023_df])
matches_22to23.partition_data(1)

## Initializing Elo Models

Elo ratings work by assigning all players a base score (provided below as a parameter to the EloModel class, by default set to 1500), and iteratively updating their score after each match played. The gain (or loss) is proportional to the difference in points won and expectation. For a single match, points are awarded as $1$ for a win, and $0$ for a loss. 

Expected points is computed as a function of a player's and his opponent's rating, as

$$
    E_A = \frac{Q_A}{Q_A + Q_B}
$$

for $Q_A = 10^{R_A / D}$, and $Q_B = 10^{R_B / D}$ for player ratings $R_A, R_B$ and model parameter $D$ (by default set to $800$ in the model). 

It follows that the score increment is computed as 

$$
    \Delta R_A = K(R_A)(S_A - E_A)
$$

For model parameter $K(x)$ as the coefficient scaler, and $S_A$ describing the point system above. The coefficient is given as a function of rating since it usually decreases for higher rated players. 

In [2]:
# Vanilla Elo model
vanilla_elo = EloModel(matches_22to23, 1500.0)

# Elo Model with boosted initialization to check convergence
heurInit_elo = EloModel(matches_22to23, 1500.0, init_heuristics = init_heur1)

In [3]:
# Looking at base ratings from boosted initialization
heurInit_elo.ratings.sort_values('rating', ascending=False)

,name,rating,games
id,,,
104925,Novak Djokovic,1801.331067,0
207989,Carlos Alcaraz,1801.331067,0
104745,Rafael Nadal,1713.441044,0
106421,Daniil Medvedev,1704.291942,0
106401,Nick Kyrgios,1695.246656,0
...,...,...,...
103852,Feliciano Lopez,1220.411998,0
106378,Kyle Edmund,1220.411998,0
126340,Viktor Durasovic,1198.668933,0


In [4]:
vanilla_elo.update_elo(matches_22to23.partitioned_data['Training'])
vanilla_elo.ratings.sort_values(by=['rating'], ascending=False)

,name,rating,games
id,,,
207989,Carlos Alcaraz,1610.197169,121
104925,Novak Djokovic,1599.859224,87
106421,Daniil Medvedev,1589.004201,120
206173,Jannik Sinner,1587.216994,114
208029,Holger Rune,1575.125314,111
...,...,...,...
202358,Chun Hsin Tseng,1463.560861,17
124079,Pedro Martinez,1458.055640,59
104269,Fernando Verdasco,1456.018112,21


In [5]:
heurInit_elo.update_elo(matches_22to23.data)
heurInit_elo.ratings.sort_values(by=['rating'], ascending=False)

,name,rating,games
id,,,
104925,Novak Djokovic,1614.869077,91
207989,Carlos Alcaraz,1602.491120,126
106401,Nick Kyrgios,1588.335154,52
106421,Daniil Medvedev,1580.100703,122
104745,Rafael Nadal,1575.081183,52
...,...,...,...
105613,Norbert Gombos,1311.001424,11
106378,Kyle Edmund,1298.861091,12
105967,Henri Laaksonen,1298.350876,17


In [5]:
def get_players(matches: pd.DataFrame):
    winner_ids = matches[['winner_id', 'winner_name']].drop_duplicates().rename(columns={'winner_id': 'id', 'winner_name': 'name'})
    loser_ids = matches[['loser_id', 'loser_name']].drop_duplicates().rename(columns={'loser_id': 'id', 'loser_name': 'name'})
    
    players = pd.concat([winner_ids, loser_ids]).drop_duplicates().set_index('id')

    return players



training_df = matches_22to23.partitioned_data['Training']
testing_df = matches_22to23.partitioned_data['Testing']
training_players = get_players(training_df)
testing_players = get_players(testing_df)

## Evaluating Models

Ratings are evaluated on a test tournament across three metrics:

- Cross-entropy (CE)
- Accuracy
- Brier Score (BS)

In [2]:
# Loading evaluation method
from elo_model import evaluate_EloModel

This implementation was a bit of an oversight, and doesn't make much sense as a stand-alone function. Model evaluation should be built into the EloModel class.

In [3]:
heurInit_eval = evaluate_EloModel(matches_22to23, init_heuristics = init_heur1)
heurInit_eval

{'N': 55,
 'CE': 0.6596622237202288,
 'Accuracy': 0.6545454545454545,
 'BS': 0.23349345317196832}

In [4]:
vanilla_eval = evaluate_EloModel(matches_22to23)
vanilla_eval

{'N': 55,
 'CE': 0.6599451564243387,
 'Accuracy': 0.6727272727272727,
 'BS': 0.2336053129324581}

In [9]:
from sklearn.metrics import log_loss

winners_OHE = np.tile([1, 0], (55, 1))
random_preds = np.tile([0.5, 0.5], (55, 1))

random_CE = log_loss(winners_OHE, random_preds)
print(random_CE)

0.6931471805599455


In [10]:
def atp_points_winner(matches: pd.DataFrame):
    N = matches.shape[0]
    predicted_wins = matches[matches['winner_rank_points'] > matches['loser_rank_points']].shape[0]

    return predicted_wins / N

atp_points_winner(matches_22to23.partitioned_data['Testing'])

0.6909090909090909

In [3]:
elo_rankings = EloModel(matches_22to23.data, 1500.0)

In [23]:
master_df = matches_22to23.data

In [4]:
elo_rankings.update_elo(matches_22to23.data)
elo_rankings.ratings.sort_values(by=['rating'], ascending=False)

,name,rating,games
id,,,
104925,Novak Djokovic,1615.337511,158
207989,Carlos Alcaraz,1610.483807,216
106421,Daniil Medvedev,1586.839918,186
100644,Alexander Zverev,1581.374282,130
206173,Jannik Sinner,1578.832783,176
...,...,...,...
202358,Chun Hsin Tseng,1463.560861,4
124079,Pedro Martinez,1458.055640,44
104269,Fernando Verdasco,1456.018112,10


In [8]:
master_df = matches_22to23.data
# master_df.groupby(['tourney_id'])['tourney_id']
master_df['tourney_id'].unique()[-3: -1]

array(['2023-0421', '2023-0422'], dtype=object)